# How to find a YouTube channel Id?

One way to get a channel Id is to go through the source code of youtube channel page and look for Id. [You can check this post](https://stackoverflow.com/questions/14366648/how-can-i-get-a-channel-id-from-youtube).   
Now imagine that you want to retrieve the IDs for a list of youtube channels, which can include one, two or several hundred youtube channels.  Then you should look for a better solution.   
An alternative solution is to use the YouTube API, if in the first place we need the id to use it, but we can work around to get the id of the channel, using the [`search`](https://developers.google.com/youtube/v3/guides/implementation/search) requests and the name of the channel. Drawback of this approach, it can return more than one result because the names of the channel are not necessarily unique. In addition, each search request costs 100 units, which can be an issue considering that the quota limit is 10,000 units per day ([quota usage](https://developers.google.com/youtube/v3/getting-started?hl=en#quota)).
 


In [1]:
from googleapiclient.discovery import build
import pandas as pd
import json
from dotenv import load_dotenv

## Load the data

First, we load a dataset of the youtube channels which we are looking for the IDs. The dataset was scraped from [Top Programmer Guru](https://noonies.tech/award/top-programming-guru) web page using the framework [`selenium`](https://github.com/SeleniumHQ/). 

In [ ]:
channelList = pd.read_csv('/data/top-programming-guru.csv', index_col=0)

In [ ]:
channelList.head()

,channelName,link
0,Programming with Mosh,https://www.youtube.com/c/programmingwithmosh/...
1,Traversy Media,https://www.youtube.com/user/TechGuyWeb
2,Corey Schafer,https://www.youtube.com/user/schafer5
3,Tech With Tim,https://m.youtube.com/channel/UC4JX40jDee_tINb...
4,Krish Naik,https://www.youtube.com/user/krishnaik06/playl...


In [ ]:
channelList.rename({'link':'url'},inplace=True, axis=1)

In [ ]:
channelList.head()

,channelName,url
0,Programming with Mosh,https://www.youtube.com/c/programmingwithmosh/...
1,Traversy Media,https://www.youtube.com/user/TechGuyWeb
2,Corey Schafer,https://www.youtube.com/user/schafer5
3,Tech With Tim,https://m.youtube.com/channel/UC4JX40jDee_tINb...
4,Krish Naik,https://www.youtube.com/user/krishnaik06/playl...


In [ ]:
# we store channels title in list
channels = channelList.channelName.values

In [ ]:
len(channels)

71

In [ ]:
# display the first element in the channel list
for channel in channels:
  print(channel)
  break

Programming with Mosh


## Load the API key

In [ ]:
load_dotenv()
API_KEY = os.getenv('api_key')
youtube = build('youtube', 'v3', developerKey=API_KEY)

In [ ]:
request = youtube.search().list(
        part="snippet",
        maxResults=1, # limit the search to one result
        q="Programming with Mosh",
        type="channel" # we going to look only for a youtube channels
    )
response = request.execute()
print(response)

In [ ]:
response = request.execute()
print(response)

In [ ]:
json.dumps(response['items'][0]['snippet']['channelId'], indent=4)

'"UCWv7vMbMWH4-V0ZXdmDpPBA"'

### Display the result of the first request

In [ ]:
for item in response['items']:
  print(item['snippet']['channelId'])
  print(item['snippet']['title'])
  print(item['id']['kind'])
  print('*' * 10)

UCWv7vMbMWH4-V0ZXdmDpPBA
Programming with Mosh
youtube#channel


### Initialize dictionary to store the results

In [ ]:
data = {'channelName': [], 'title': [], 'id':[], 'kind':[]}

In [ ]:
id = response['items'][0]['snippet']['channelId']
title = response['items'][0]['snippet']['title']
kind = response['items'][0]['id']['kind']

In [ ]:
channel

'Programming with Mosh'

In [ ]:
data['channelName'].append(channel)
data['title'].append(title)
data['id'].append(id)
data['kind'].append(kind)

In [ ]:
data

{'channle': ['Programming with Mosh'],
 'id': ['UCWv7vMbMWH4-V0ZXdmDpPBA'],
 'kind': ['youtube#channel'],
 'title': ['Programming with Mosh']}

### Retrive the Id for each channel in dataset

In [ ]:
for channel in channels[1:]:

  request = youtube.search().list(
        part="snippet",
        maxResults=1,
        q=channel,
        type="channel"
    )
  
  response = request.execute()

  id = response['items'][0]['snippet']['channelId']
  title = response['items'][0]['snippet']['title']
  kind = response['items'][0]['id']['kind']

  data['channelName'].append(channel)
  data['title'].append(title)
  data['id'].append(id)
  data['kind'].append(kind)

In [ ]:
data

In [ ]:
df = pd.DataFrame.from_dict(data)

In [ ]:
df.head()

,channle,title,id,kind
0,Programming with Mosh,Programming with Mosh,UCWv7vMbMWH4-V0ZXdmDpPBA,youtube#channel
1,Traversy Media,Traversy Media,UC29ju8bIPH5as8OGnQzwJyA,youtube#channel
2,Corey Schafer,Corey Schafer,UCCezIgC97PvUuR4_gbFUs5g,youtube#channel
3,Tech With Tim,Tech With Tim,UC4JX40jDee_tINbkjycV4Sg,youtube#channel
4,Krish Naik,Krish Naik,UCNU_lfiiWBdtULKOw6X0Dig,youtube#channel


In [ ]:
df['kind'].unique()

array(['youtube#channel'], dtype=object)

Let's check if there is a difference between the channel name and the title.

In [ ]:
(df.channelName != df.title).sum()

23

Let's display the rows where the `channelName` and the `title` are different.

In [ ]:
df[df.channelName != df.title]

,channle,title,id,kind
9,programming Hero,Programming Hero,UCStj-ORBZ7TGK1FwtGAUgbQ,youtube#channel
10,naveen Reddy,Telusko,UC59K-uG2A5ogwIrHw4bmlEg,youtube#channel
11,Code Basics,codebasics,UCh9nVJoWXmFb7sLApWGcLPQ,youtube#channel
12,Maximilian Schwarzmüller,Academind,UCSJbGtTlrDami-tDGPUV9-w,youtube#channel
19,Code with Harry,CodeWithHarry,UCeVMnSShP_Iviwkknt83cww,youtube#channel
22,Tim Corey,IAmTimCorey,UC-ptWR16ITQyYOglXyQmpzw,youtube#channel
24,WesBos,Wes Bos,UCoebwHSTvwalADTJhps0emA,youtube#channel
25,Gajesh S. Naik,Gajesh S Naik,UC7PWnwwqMSqAXQkKXqxRkMw,youtube#channel
34,Scott Hansellman,Scott Hanselman,UCL-fHOdarou-CR2XUmK48Og,youtube#channel
38,Joma,Joma Tech,UCV0qA-eDDICsRR9rPcnG7tw,youtube#channel


We get these results because in the web page from where the data was collected, it some they used the name of the owner of the youtube channel instead of the official title, or they did not respect the same case as in the title of the channel.   
We going to store data in `csv` file and check agin the difference between the channel name and the title but this time ignoring case.

In [ ]:
df.to_csv('data/channelsID.csv')

In [4]:
data = pd.read_csv('data/channelsID.csv', index_col=0)

In [ ]:
data.head()

In [9]:
(data.channle.str.lower() != data.title.str.lower()).sum()

14

In [10]:
data[data.channle.str.lower() != data.title.str.lower()]

,channle,title,id,kind
10,naveen Reddy,Telusko,UC59K-uG2A5ogwIrHw4bmlEg,youtube#channel
11,Code Basics,codebasics,UCh9nVJoWXmFb7sLApWGcLPQ,youtube#channel
12,Maximilian Schwarzmüller,Academind,UCSJbGtTlrDami-tDGPUV9-w,youtube#channel
19,Code with Harry,CodeWithHarry,UCeVMnSShP_Iviwkknt83cww,youtube#channel
22,Tim Corey,IAmTimCorey,UC-ptWR16ITQyYOglXyQmpzw,youtube#channel
24,WesBos,Wes Bos,UCoebwHSTvwalADTJhps0emA,youtube#channel
25,Gajesh S. Naik,Gajesh S Naik,UC7PWnwwqMSqAXQkKXqxRkMw,youtube#channel
34,Scott Hansellman,Scott Hanselman,UCL-fHOdarou-CR2XUmK48Og,youtube#channel
38,Joma,Joma Tech,UCV0qA-eDDICsRR9rPcnG7tw,youtube#channel
42,Programming With ERik,Program With Erik,UCshZ3rdoCLjDYuTR_RBubzw,youtube#channel


Now we only have `14` rows left to verify. Quick check in Youtube and we see that there is only one error, it is in row 54. Let's correct the error and save the data.

In [11]:
channle_54 = {'title': 'IIMB Inventors Inventing Machine Business', 'id':'UCWym_j-OGCzIfz8K4vXtU6g'}

In [15]:
data.loc[54, 'title'] = 'IIMB Inventors Inventing Machine Business'
data.loc[54, 'id'] = 'UCWym_j-OGCzIfz8K4vXtU6g'

In [16]:
data.loc[54]

channle                                  Prajwal NH 
title      IIMB Inventors Inventing Machine Business
id                          UCWym_j-OGCzIfz8K4vXtU6g
kind                                 youtube#channel
Name: 54, dtype: object

In [17]:
data.to_csv('data/channelsID.csv')